# Inputs

In [18]:
import os
import pandas as pd
import numpy as np
import scipy.io
from scipy.io import loadmat, savemat
from pathlib import Path

import json # deprecated
import pickle # alternative 

path_main = os.getcwd()
path_inputs = os.path.join(path_main, '02 Inputs')

# General Variables
m = 8760  # [h] #hours per year
r = 0.08  # [-] cost of capital

alpha_s = 0.0  # [-] taxfoundation.org, CA 0.084
alpha_f = 0.21  # [-] taxfoundation.org

# Investment Tax Credits (45V and 48C) - mutally 
ITC = 0.0  # 0.3;
ITC_capdisc = 0  # 0.5;

PTC_h_pretax = 3 # 3.0
PTC_h = PTC_h_pretax # / (1 - alpha_s - alpha_f * (1 - alpha_s)) # Adjustment in main file now as different levels
PTC_h_life = 10

PTC_grid_pretax = 3 # 3 Put as 0 for now
PTC_grid = PTC_grid_pretax / (1 - alpha_s - alpha_f * (1 - alpha_s))

# PTC H2 on kwh basis for conversion - not needed for now
# PTC_h_kwh = PTC_h * eta_c 
# print(PTC_h_kwh)

# Renewables
PTC_pretax = 0.015  # [$/kwh] // 0.03 max // 0.015 all renewables (previously!)
PTC = PTC_pretax / (1 - alpha_s - alpha_f * (1 - alpha_s)) # tax adjusted
PTC_life = 10

# PTC_renew = 0.026 # [$/kg]

# Lifetime
T_max = 20 

# Create a time vector
T = list(range(0, T_max + 1))  # years

# PTC should be paid starting only in t = 1
PTC_vector = [0 if year == 0 else (PTC if year <= PTC_life else 0) for year in range(T_max + 1)]
PTC_h_vector = [0 if year == 0 else (PTC_h if year <= PTC_h_life else 0) for year in range(T_max + 1)]
PTC_grid_vector = [0 if year == 0 else (PTC_grid if year <= PTC_h_life else 0) for year in range(T_max + 1)]

# Depreciation
# 1: No depreciation schedule
# 2: 5-year MACRS DDB depreciation schedule
# 3: 20-year 150%-declining balance depreciation schedule
# 4: MACRS with 50% bonus 1st year
# 5: 100% bonus depreciation
d_schedules = np.zeros((31, 5))  # [-] max 30y lifetime + year 0, 5 schedules
d_schedules[1:7,1] = [0.2000, 0.3200, 0.1920, 0.1152, 0.1152, 0.0576]
d_schedules[1:21, 2] = [0.07500000000000000, 0.06937500000000000, 
                        0.06417187500000000, 0.05935898437500000, 
                        0.05490706054687500, 0.05078903100585940, 
                        0.04697985368041990, 0.04457063041475740, 
                        0.04457063041475740, 0.04457063041475740, 
                        0.04457063041475740, 0.04457063041475740, 
                        0.04457063041475740, 0.04457063041475740, 
                        0.04457063041475740, 0.04457063041475740, 
                        0.04457063041475740, 0.04457063041475740, 
                        0.04457063041475740, 0.04457063041475740]
d_schedules[0:7, 3] = [0.5000, 0.1000, 0.1600, 0.0960, 0.0576, 0.0576, 0.0288]
d_schedules[0, 4] = 1

# print(d_schedules) # Test if done correctly

# d_type_s = 3  # [type] Type of depr. schedule in state
d_type_f = 3  # [type] Type of depr. schedule for federal purposes

# Initialize depreciation array
d_i = np.zeros(T_max+1)

# Assign depreciation values based on the chosen schedule - Deleted all +1 ranges
d_i[0:T_max+1] = d_schedules[0:T_max+1, d_type_f-1]

## Electrolysis Inputs

In [21]:
# Simpler Version

w_o = 0.10  # [$/kg] estimation of water cost

# # Technology Variables - for now keep 3 technologies
# lifetime = np.array([20, 20, 20])  # [years] assumption
# x_h = np.array([0.01, 0.01, 0.016])  # [-] Sources: doe2019report, ﻿bloomberg2019hydrogen (Table 7)
eta_c = np.array([0.0184568181818182, 0.0188545454545455, 0.0209544349383005])  # [kg/kWh] Source: 230323 Cost and efficiency forecast.xlsx

F_h = np.array([0.025, 0.03, 0.03])  # % of SP_h [$/kW] Source: Buttler/Spliethoff (2018), Table 3
# SP_h = np.array([901.521562846846, 1076.4902191004, 2314.24386978137])  # [$/kW] Source: 230323 Cost and efficiency forecast

SP_h = np.array([1000, 1500, 2314.24386978137]) # Test with higher input prices # Range of $1,300-$1,700/kW in newest NREL 2024 report (installed final costs full system)

# Deprecated
# cost_file = os.path.join(path_inputs, '230323 Cost and efficiency forecast.xlsx')  # [-] Multiplicative change to SP_h in 2020 per technology
# forecasts = pd.read_excel(cost_file, sheet_name='Export') # Could simply multiply our etas and SPs with the forecast factors


## Renewable Inputs 

In [24]:
# Renewable Inputs - Could use SAM inputs directly?

# Wind

SP_e_w = 1609 # LCOE tool: # Input from 2024 for Texas Previous: 1156 $/kW # or put average from Land-based wind report 2023: 1366.7 
F_ei_w = 12.840 # LCOE tool update
W_ei_w = 0.0013 # LCOE tool
k_e_w = 1.00 

# Solar
SP_e_s = 1133 # LCOE tool update 2024
F_ei_s = 5.133 # LCOE tool update 2024
W_ei_s = 0.0005 # LCOE tool
k_e_s = 1.00 

x_e = 0.01 # Just in case we want to split up degradation 

# Deprecated - change in the export
wind_input = os.path.join(path_inputs, 'wind_CFs.xlsx') # wind_CFs // wind_CFs_2 -> HRRR - Median (one plant), Best (all median plants - averaged)
wind = pd.read_excel(wind_input, sheet_name='Texas_best')

CF_wind = wind["CF"]

solar_input = os.path.join(path_inputs, 'solar_CFs.xlsx') # solar_CFs = Test values from SAM // solar_CFs_2 = Process as in word
solar = pd.read_excel(solar_input, sheet_name='Texas_best')

CF_solar = solar["CF"]

# CF_wind.head(10)

In [26]:
wind_input = os.path.join(path_inputs, 'wind_CFs.xlsx') # Wind_CFs // wind_CFs_2
solar_input = os.path.join(path_inputs, 'solar_CFs.xlsx') # Solar_CFs // solar_CFs_2

# Load data from Excel sheets - maybe add more later or change approach
def load_cf_data(input_path, sheet_names):
    data = {}
    for sheet_name in sheet_names:
        df = pd.read_excel(input_path, sheet_name=sheet_name)
        data[sheet_name] = df['CF'].values  
    return data

# Updated list of sheet names - scenario + state 

sheet_names = [
    'Texas_best', 'Texas_median', 'Texas_worst', 
    'California_best', 'California_median', 'California_worst',
    'Germany_best', 'Germany_median', 'Germany_worst'
]

# Load data
wind_data = load_cf_data(wind_input, sheet_names)
solar_data = load_cf_data(solar_input, sheet_names)

# Verify
for source, data in [('Wind', wind_data), ('Solar', solar_data)]:
    print(f"{source} Data:")
    for sheet, cf_values in data.items():
        print(f"  {sheet}: {cf_values[:10]}")  # Test


Wind Data:
  Texas_best: [0.902944   0.91941767 0.89932367 0.86326967 0.836188   0.823582
 0.80637233 0.79362167 0.77660633 0.74781833]
  Texas_median: [0.664007   0.75439    0.75085233 0.68463433 0.59599733 0.52882367
 0.54314767 0.595061   0.64902233 0.66273933]
  Texas_worst: [0.04637967 0.042688   0.04095267 0.04841267 0.06325967 0.082462
 0.11638967 0.17100567 0.23042267 0.26252967]
  California_best: [0.00515133 0.00935133 0.031447   0.057251   0.06903767 0.07868933
 0.09268833 0.125758   0.164246   0.18765733]
  California_median: [0.06830367 0.032947   0.021599   0.027147   0.03429133 0.03591
 0.03440233 0.04071667 0.04846467 0.051321  ]
  California_worst: [0.00605667 0.00046733 0.00189067 0.006818   0.00719233 0.00543333
 0.00557233 0.011552   0.029151   0.056153  ]
  Germany_best: [0.51333    0.467782   0.47898067 0.480406   0.452257   0.47419033
 0.49701667 0.52346133 0.53778367 0.50719367]
  Germany_median: [0.25258633 0.24820933 0.243471   0.25259467 0.278962   0.325554
 

In [28]:
# Combined data dictionary
combined_renewable_data = {
    'Texas': {},
    'California': {},
    'Germany': {}
}

# Combine data for each state and scenario
scenarios = ['best', 'median', 'worst']
for state in combined_renewable_data.keys():
    for scenario in scenarios:
        wind_key = f"{state}_{scenario}"
        solar_key = f"{state}_{scenario}"
        combined_renewable_data[state][scenario] = {
            'wind': wind_data[wind_key],
            'solar': solar_data[solar_key]
        }

# Save combined data for use in optimization
with open(os.path.join(path_inputs, 'combined_renewables.pkl'), 'wb') as f:
    pickle.dump(combined_renewable_data, f)

## Grid Inputs

In [31]:
# Wholesale Electricity Prices Input (hourly) - Still see different files!
               
price_file = os.path.join(path_inputs, 'Power_prices.xlsx')  # [$/kWh] 
# Decide on Jurisdiction: DE (TBD), TX (ERCOT_HubAvg_2010-2020.xlsx), CA (CAISO_HubAvg_2010-2020)

# Function to load electricity prices for a given region
def load_electricity_prices(region):
    sheet_name = region  # Each region has its own sheet
    try:
        data = pd.read_excel(price_file, sheet_name=sheet_name)
        print(f"Data successfully loaded for {sheet_name}")
        return data
    except Exception as e:
        print(f"An error occurred while loading the data: {e}")
        return None

# temp = pd.read_excel(price_file, sheet_name='Import')
p_s_caiso = load_electricity_prices('CALIFORNIA')
p_s_ercot = load_electricity_prices('TEXAS')
p_s_de = load_electricity_prices('DE')

p_s_t_CA = p_s_caiso["Avg4"] # Change in case to singular year (without "") or different average -> currently very low power prices (2016-2020)
p_s_t_TX = p_s_ercot["Avg4"] # "Avg1" before (2016-2020) - lowest vector # single years e.g. 2023
p_s_t_DE = p_s_de["Avg4"] # "Avg4" includes 2015-2023 day ahead hourly prices

print(p_s_t_TX)

w_grid = 0.00788008 # + 0.2 # [$/kWh] Markup on buying price, Griddy and CenterPoint Charge
p_b_t_CA = p_s_t_CA + w_grid * 1.01
p_b_t_TX = p_s_t_TX + w_grid * 1.01 #### !!! Trial with increased price for grid transfer compared to renewable power. -> only marginal 1%?
p_b_t_DE = p_s_t_DE + w_grid * 1.01 # + 0.02 # Possible increase as Netzentgelt is higher in Germany


Data successfully loaded for CALIFORNIA
Data successfully loaded for TEXAS
Data successfully loaded for DE
0       0.019318
1       0.020015
2       0.018433
3       0.017794
4       0.017411
          ...   
8755    0.026408
8756    0.023050
8757    0.022013
8758    0.019912
8759    0.018877
Name: Avg4, Length: 8760, dtype: float64


In [32]:
# Carbon intensity of the grid

carbon_file = os.path.join(path_inputs, 'Carbon_Intensity_Data.xlsx')

# load CI for a given year
def load_carbon_intensity(year):
    sheet_name = f'CI_{year}'
    try:
        data = pd.read_excel(carbon_file, sheet_name=sheet_name)
        print(f"Data successfully loaded for {sheet_name}")
        return data
    except Exception as e:
        print(f"An error occurred while loading the data: {e}")
        return None

# Carbon
carbon_intensity = load_carbon_intensity("Avg_1") # Or enter year: 2019 - 2023 (without "") # So far "Avg_1" over 4 years
print(carbon_intensity.head())

Data successfully loaded for CI_Avg_1
   period  CO2i_ERCO_D  CO2i_CISO_D  CO2i_MISO_D  CO2i_PJM_D  CO2i_SWPP_D  \
0       1     0.361845     0.305881     0.470292    0.336749     0.484467   
1       2     0.321227     0.306958     0.455548    0.335613     0.352915   
2       3     0.314004     0.306544     0.455661    0.335191     0.349809   
3       4     0.305593     0.301394     0.454429    0.333849     0.350045   
4       5     0.296925     0.298237     0.452552    0.333486     0.350713   

   CO2i_NYIS_D        DE  
0     0.214394  0.279424  
1     0.210667  0.272305  
2     0.208216  0.270726  
3     0.208587  0.275714  
4     0.207941  0.276208  


# Gather all variables and export

In [42]:
# Collect Variables 
variables = {
    'm': m,
    'lifetime': T_max,
    'r': r,
    'alpha_s': alpha_s,
    'alpha_f': alpha_f,
    'ITC': ITC,
    'ITC_capdisc': ITC_capdisc,
    'PTC': PTC,
    'PTC_life': PTC_life,
    'PTC_vector': PTC_vector,
    'PTC_h_vector': PTC_h_vector,
    'PTC_grid_vector': PTC_grid_vector,
    'x_h': x_h,
    'x_e': x_e,
    'eta_c': eta_c,
    'w_o': w_o,
    'SP_h': SP_h,
    'F_h': F_h,
    'd_schedules': d_schedules,
    'd_type_s': d_type_s,
    'd_type_f': d_type_f,
    'p_s_t_CA': p_s_t_CA,
    'p_s_t_TX': p_s_t_TX,
    'p_s_t_DE': p_s_t_DE,
    'p_b_t_CA': p_b_t_CA,
    'p_b_t_TX': p_b_t_TX,
    'p_b_t_DE': p_b_t_DE,
    'w_grid': w_grid,
    'forecasts': forecasts,
    'T': T,
    "d_i": d_i,
    "SP_e_w": SP_e_w,
    "F_ei_w": F_ei_w,
    "W_ei_w": W_ei_w,
    "k_e_w": k_e_w,
    "SP_e_s": SP_e_s,
    "F_ei_s": F_ei_s,
    "W_ei_s": W_ei_s,
    "k_e_s": k_e_s,
    "CF_wind": CF_wind,
    "CF_solar":CF_solar,
    "PTC_pretax":PTC_pretax,
    # "PTC_renew":PTC_renew,
    "PTC_h_pretax": PTC_h_pretax,
    "PTC_h_life": PTC_h_life,
    # "PTC_h_kwh": PTC_h_kwh,
    "PTC_h": PTC_h,
    "PTC_grid": PTC_grid,
    #'EIA_exp': price_exp['EIA_Exp'], # Excluded for now
    #'flat': price_exp['Flat'],
    #'own_exp': price_exp2,
    "carbon_intensity": carbon_intensity
}

# Save Variables

with open(os.path.join(path_inputs, 'variables.pkl'), 'wb') as handle:
    pickle.dump(variables, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Cross-Check type (varies for different storing formats)
for key, value in variables.items():
    print(f"{key}: {type(value)}")

time_horizon: <class 'int'>
m: <class 'int'>
fxrate: <class 'float'>
state_cost_idx: <class 'int'>
lifetime: <class 'int'>
r: <class 'float'>
alpha_s: <class 'float'>
alpha_f: <class 'float'>
ITC: <class 'float'>
ITC_capdisc: <class 'int'>
PTC: <class 'float'>
PTC_life: <class 'int'>
PTC_vector: <class 'list'>
PTC_h_vector: <class 'list'>
PTC_grid_vector: <class 'list'>
x_h: <class 'numpy.ndarray'>
x_e: <class 'float'>
eta_c: <class 'numpy.ndarray'>
w_o: <class 'float'>
SP_h: <class 'numpy.ndarray'>
F_h: <class 'numpy.ndarray'>
d_schedules: <class 'numpy.ndarray'>
d_type_s: <class 'int'>
d_type_f: <class 'int'>
p_s_t_CA: <class 'pandas.core.series.Series'>
p_s_t_TX: <class 'pandas.core.series.Series'>
p_s_t_DE: <class 'pandas.core.series.Series'>
p_b_t_CA: <class 'pandas.core.series.Series'>
p_b_t_TX: <class 'pandas.core.series.Series'>
p_b_t_DE: <class 'pandas.core.series.Series'>
w_grid: <class 'float'>
forecasts: <class 'pandas.core.frame.DataFrame'>
T: <class 'list'>
d_i: <class 'n

In [43]:
# all_variables = locals()

# # Print all local variables
# for name, value in all_variables.items():
#     print(f"{name}: {value}")

In [15]:
# locals.type()